# Pre-trained TSN Models

### Reference pages: 

https://cv.gluon.ai/install/install-more.html

https://cv.gluon.ai/build/examples_action_recognition/demo_tsn_ucf101.html


### Environment Set Up

Created new environment (python version 3.8.19), and installed lower torch version directly from the source

pip install https://download.pytorch.org/whl/cpu/torch-1.6.0-cp38-none-macosx_10_9_x86_64.whl    

This environment also needs lower numpy version --> pandas doesn't work


In [16]:
pip install numpy==1.19.3

  Using cached numpy-1.19.3-cp38-cp38-macosx_10_9_x86_64.whl.metadata (2.0 kB)
Using cached numpy-1.19.3-cp38-cp38-macosx_10_9_x86_64.whl (15.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matplotlib 3.7.5 requires numpy<2,>=1.20, but you have numpy 1.19.3 which is incompatible.
pandas 2.0.3 requires numpy>=1.20.3; python_version < "3.10", but you have numpy 1.19.3 which is incompatible.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 1.19.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
#import matplotlib.pyplot as plt
import mxnet as mx
from mxnet import gluon, nd, image
from mxnet.gluon.data.vision import transforms
from gluoncv.data.transforms import video
from gluoncv import utils
from gluoncv.model_zoo import get_model
import numpy as np

/Users/hp/opt/anaconda3/envs/viz_project/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/hp/opt/anaconda3/envs/viz_project/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/hp/opt/anaconda3/envs/viz_project/lib/python3.8/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.1` and `torch==1.6.0` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


In [3]:
from IPython.display import clear_output, display, Image
import os
import subprocess
import re
import cv2

In [4]:
video_path = "/Users/hp/viz_project_trials/video/sample_video_pop_estimation.mp4"

output_dir = os.path.join("/Users/hp/viz_project_trials/content", "extracted_frames")
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
  print("Error opening video!")
  exit()

# frames per second
fps = int(cap.get(cv2.CAP_PROP_FPS))
# total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

frame_count = 1
actual_count = 0

# Read until video is completed
while True:
  ret, frame = cap.read()

  if not ret:
    print("Can't receive frame (stream end?). Exiting...")
    break

  # Save frame as numbered image
  if frame_count % fps == 0:
    image_path = os.path.join(output_dir, f"{frame_count}.jpg")
    cv2.imwrite(image_path, frame)
    actual_count += 1

  frame_count += 1

  if frame_count > total_frames:
    break

cap.release()

print(f"Extracted {actual_count} frames to {output_dir}")

Extracted 308 frames to /Users/hp/Population-Estimation-using-Street-Cam-Footage/content/extracted_frames


Try one image

In [5]:
"""
# Path to the image file you want to process
image_path = "/Users/hp/Population-Estimation-using-Street-Cam-Footage/content/extracted_frames/112.jpg"

# Open the image
img = image.imread(image_path)
"""


Get Image directory

In [10]:
# Path to your extracted frames directory
frames_dir = "/Users/hp/viz_project_trials/content/extracted_frames"

# Get list of image files in the directory
image_files = sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.jpg')])

In [11]:
transform_fn = transforms.Compose([
    video.VideoCenterCrop(size=224),
    video.VideoToTensor(),
    video.VideoNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
#img_list = transform_fn([img.asnumpy()])

In [12]:
net = get_model('vgg16_ucf101', nclass=101, pretrained=True)

In [15]:
output_file = "/Users/hp/Population-Estimation-using-Street-Cam-Footage/output.txt"

with open(output_file, "w") as f:
    for image_path in image_files:
        # Open the image
        img = image.imread(image_path)

        img_list = transform_fn([img.asnumpy()])

        pred = net(nd.array(img_list[0]).expand_dims(axis=0))

        classes = net.classes
        topK = 5
        ind = nd.topk(pred, k=topK)[0].astype('int')

        # Store labels and probabilities in lists
        labels = []
        probabilities = []

        for i in range(topK):
            label = classes[ind[i].asscalar()]
            probability = nd.softmax(pred)[0][ind[i]].asscalar()
            labels.append(label)
            probabilities.append(probability)

        # Write to the output file
        f.write(f"Image: {os.path.basename(image_path)}\n")
        for label, prob in zip(labels, probabilities):
            f.write(f"\t{label}: {prob}\n")
        f.write("\n")

print(f"Labels, probabilities, and image names saved to {output_file}")

Labels, probabilities, and image names saved to /Users/hp/Population-Estimation-using-Street-Cam-Footage/output.txt
